# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [3]:
#Definition of the tables.
import pandas as pd

# Table and definitions for a multiplayer game
data = [
    {"table": "players", "definition": "Stores player details, including username, email, registration date, and level."},
    {"table": "matches", "definition": "Contains match records, including match ID, date, game mode, and duration."},
    {"table": "player_matches", "definition": "Links players to matches, storing player ID, match ID, score, and performance metrics."},
    {"table": "leaderboards", "definition": "Maintains ranking details of players based on points and achievements."},
    {"table": "achievements", "definition": "Tracks in-game achievements unlocked by players, including achievement type and date earned."},
]
df = pd.DataFrame(data)
print(df)

            table                                         definition
0         players  Stores player details, including username, ema...
1         matches  Contains match records, including match ID, da...
2  player_matches  Links players to matches, storing player ID, m...
3    leaderboards  Maintains ranking details of players based on ...
4    achievements  Tracks in-game achievements unlocked by player...


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [6]:
print(text_tables)

players: Stores player details, including username, email, registration date, and level.
matches: Contains match records, including match ID, date, game mode, and duration.
player_matches: Links players to matches, storing player ID, match ID, score, and performance metrics.
leaderboards: Maintains ranking details of players based on points and achievements.
achievements: Tracks in-game achievements unlocked by players, including achievement type and date earned.


In [7]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [8]:
#Creating the prompt, with the user questions and the tables definitions.
userQ = "Who are the top 10 players with the highest total score in ranked matches this season?"
pqt1 = prompt_question_tables.format(tables=text_tables, question=userQ)

In [9]:
print(return_OAI(pqt1))

{
    "tables": ["players", "matches", "player_matches"]
}


In [10]:
userQ = "Which player has completed the most achievements?"
pqt3 = prompt_question_tables.format(tables=text_tables, question=userQ)

In [11]:
print(return_OAI(pqt3))

{
    "tables": ["players", "achievements"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [12]:
# Table definitions for a battle royale game
data = [
    {"table": "players", "definition": "Stores player details, including username, rank, and total matches played."},
    {"table": "matches", "definition": "Contains battle royale match details such as match ID, start time, and total players."},
    {"table": "player_matches", "definition": "Tracks player participation in matches, including kills, damage dealt, and survival time."},
    {"table": "weapons", "definition": "Stores information about weapons, including damage, type, and availability."},
    {"table": "player_weapons", "definition": "Links players to weapons used in matches, tracking kills and accuracy per weapon."},
]

df = pd.DataFrame(data)

text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for _, row in df.iterrows()])


In [14]:
userQ = "Which weapon has the highest average kill rate in solo matches?"
pqt01 = prompt_question_tables.format(tables=text_tables, question=userQ)
print(return_OAI(pqt01))

{
    "tables": ["weapons", "player_weapons", "player_matches"]
}


In [15]:
userQ = "Which landing zone has the highest survival rate in the first 5 minutes of a match?"
pqt02 = prompt_question_tables.format(tables=text_tables, question=userQ)
print(return_OAI(pqt02))

{
    "tables": ["matches", "player_matches"]
}


In [16]:
userQ = "Which weapon has the highest headshot accuracy across all players?"
pqt03 = prompt_question_tables.format(tables=text_tables, question=userQ)
print(return_OAI(pqt03))

{
    "tables": ["weapons", "player_weapons"]
}


In [17]:
userQ = "Which players have won the most matches in squad mode this season?"
pqt04 = prompt_question_tables.format(tables=text_tables, question=userQ)
print(return_OAI(pqt04))

{
    "tables": ["players", "matches", "player_matches"]
}


### Report on GPT-Generated SQL Table Selection for Gaming Analytics
    During testing, GPT-3.5 Turbo effectively identified relevant tables for SQL queries in Online Multiplayer Game – Player Stats and Leaderboards and Battle Royale Game – Match Analytics scenarios. 

- Key Learnings:
    - Clear Table Definitions Improve Accuracy – Well-structured definitions reduce hallucinations.

    - JSON Output Helps Parsing – Structured responses make it easier to validate and refine table selection.
